In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
#nltk.download('genesis')
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

import spacy
import re
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [2]:
class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    def convert_tag(self, tag):
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None

    def doc_to_synsets(self, doc):
        tokens = word_tokenize(doc+' ')       
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l
    
    def similarity_score(self, s1, s2, distance_type = 'path'):
        s1_largest_scores = []

        for i, s1_synset in enumerate(s1, 0):
            max_score = 0
            for s2_synset in s2:
                if distance_type == 'path':
                    score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                else:
                    score = s1_synset.wup_similarity(s2_synset)                  
                if score != None:
                    if score > max_score:
                        max_score = score
              
            if max_score != 0:
                s1_largest_scores.append(max_score)
          
        mean_score = np.mean(s1_largest_scores)
                 
        return mean_score  

def document_similarity(self,doc1, doc2):
        synsets1 = self.doc_to_synsets(doc1)
        synsets2 = self.doc_to_synsets(doc2)
        return ((self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2)

In [3]:
knn = KNN_NLC_Classifer()

file1 = open(r"C:\Users\Austin\txt_file\adapt_bulletin-adapt1-eng.txt", mode = 'r', encoding = 'utf-8-sig')
l1 = file1.read()
file1.close()

file2 = open(r"C:\Users\Austin\txt_file\81363.txt", mode = 'r', encoding = 'utf-8-sig')
l2 = file2.read()
file2.close()

print("Test Similarity Score: ", document_similarity(knn, l1, l2))

Test Similarity Score:  0.7208613785670974


The files "81363" and "Adapt_bulletin-adapt1-eng" (PDF's available on Github) have a 72% similarity.

In [4]:
file3 = open(r"C:\Users\Austin\txt_file\ccp_impactonpeople.txt", mode = 'r', encoding = 'utf-8-sig')
l3 = file3.read()
file3.close()

file4 = open(r"C:\Users\Austin\txt_file\Bruce_(2006)_AdaptingtoClimateChange_ARisk-basedGuideforONMunicipalities.txt", mode = 'r', encoding = 'utf-8-sig')
l4 = file4.read()
file4.close()

print("Test Similarity Score: ", document_similarity(knn, l3, l4))

Test Similarity Score:  0.7321147529965923


The files "ccp_impactonpeople" and "Bruce_(2006)_AdaptingtoClimateChange_ARisk-basedGuideforONMunicipalities" (PDF's available on Github) have a 73% similarity.

In [5]:
file5 = open(r"C:\Users\Austin\txt_file\public_guideline__principles_of_climate_adaptation_and_mitigation_for_engineers.txt", mode = 'r', encoding = 'utf-8-sig')
l5 = file5.read()
file5.close()

file6 = open(r"C:\Users\Austin\txt_file\Spring_Flood_Fact_Sheet.txt", mode = 'r', encoding = 'utf-8-sig')
l6 = file6.read()
file6.close()

print("Test Similarity Score: ", document_similarity(knn, l5, l6))

Test Similarity Score:  0.7144099232801115


The files "public_guideline__principles_of_climate_adaptation_and_mitigation_for_engineers" and "Spring_Flood_Fact_Sheet" (PDF's available on Github) have a 71% similarity.